### CHAPTER 8
# Data Wrangling: Join, Combine, and Reshape


In [20]:
import pandas as pd
import numpy as np

## 8.1 Hierarchical Indexing

In [21]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.814868
   2    0.583829
   3    0.405178
b  1    0.352636
   3    0.746648
c  1    0.510672
   2    0.991023
d  2    0.260703
   3    0.751050
dtype: float64

In [22]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [23]:
data["b"]

1    0.352636
3    0.746648
dtype: float64

In [24]:
data["b":"c"]

b  1    0.352636
   3    0.746648
c  1    0.510672
   2    0.991023
dtype: float64

In [25]:
data.loc[:, 2]

a    0.583829
c    0.991023
d    0.260703
dtype: float64

In [26]:
data.unstack()

,1,2,3
a,0.814868,0.583829,0.405178
b,0.352636,NaN,0.746648
c,0.510672,0.991023,NaN
d,NaN,0.260703,0.751050


In [27]:
data.unstack().stack()

a  1    0.814868
   2    0.583829
   3    0.405178
b  1    0.352636
   3    0.746648
c  1    0.510672
   2    0.991023
d  2    0.260703
   3    0.751050
dtype: float64

In [28]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [29]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [30]:
frame.index.nlevels

2

In [31]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [32]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                           ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

#### Reordering and Sorting Levels

In [33]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [34]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [35]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [36]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11